In [ ]:
# Install PyTorch
!pip install -U torch==1.5.0+cu101 torchvision==0.6.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
# Install MMCV
!pip install mmcv-full==latest+torch1.5.0+cu101 -f https://download.openmmlab.com/mmcv/dist/index.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 703.8MB 26kB/s 
     |████████████████████████████████| 6.6MB 32.7MB/s 
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.5.0+cu101 which is incompatible.
  Found existing installation: torch 1.8.1+cu101
    Uninstalling torch-1.8.1+cu101:
      Successfully uninstalled torch-1.8.1+cu101
  Found existing installation: torchvision 0.9.1+cu101
    Uninstalling torchvision-0.9.1+cu101:
      Successfully uninstalled torchvision-0.9.1+cu101
Looking in links: https://download.openmmlab.com/mmcv/dist/index.html
     |████████████████████████████████| 27.5MB 108kB/s 
     |████████████████████████████████| 194kB 10.5MB/s 


In [ ]:
!rm -rf mmsegmentation
!git clone https://github.com/open-mmlab/mmsegmentation.git 
%cd mmsegmentation
!pip install -e .
!mkdir checkpoints

Cloning into 'mmsegmentation'...
remote: Enumerating objects: 2547, done.
remote: Counting objects: 100% (104/104), done.
remote: Compressing objects: 100% (88/88), done.
remote: Total 2547 (delta 34), reused 58 (delta 15), pack-reused 2443
Receiving objects: 100% (2547/2547), 4.70 MiB | 18.36 MiB/s, done.
Resolving deltas: 100% (1811/1811), done.
/content/mmsegmentation
Obtaining file:///content/mmsegmentation
  Running setup.py develop for mmsegmentation


In [ ]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMSegmentation installation
import mmseg
print(mmseg.__version__)

1.5.0+cu101 True
0.12.0


In [ ]:
!wget https://download.openmmlab.com/mmsegmentation/v0.5/pspnet/pspnet_r101-d8_512x512_160k_ade20k/pspnet_r101-d8_512x512_160k_ade20k_20200615_100650-967c316f.pth -P /content/mmsegmentation/checkpoints

--2021-05-04 15:44:39--  https://download.openmmlab.com/mmsegmentation/v0.5/pspnet/pspnet_r101-d8_512x512_160k_ade20k/pspnet_r101-d8_512x512_160k_ade20k_20200615_100650-967c316f.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 47.254.186.225
Connecting to download.openmmlab.com (download.openmmlab.com)|47.254.186.225|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 272850621 (260M) [application/octet-stream]
Saving to: ‘/content/mmsegmentation/checkpoints/pspnet_r101-d8_512x512_160k_ade20k_20200615_100650-967c316f.pth’

pspnet_r101-d8_512x 100%[===================>] 260.21M  3.17MB/s    in 83s     

2021-05-04 15:46:03 (3.15 MB/s) - ‘/content/mmsegmentation/checkpoints/pspnet_r101-d8_512x512_160k_ade20k_20200615_100650-967c316f.pth’ saved [272850621/272850621]



In [ ]:
dataset_list = [
  '2azQ1b91cZZ_level_0', 
  '8194nk5LbLH_level_0',
  'EU6Fwq7SyZv_level_0',
  'QUCTc6BB5sX_level_0',
  'TbHJrupSAjP_level_0',
  'X7HyMhZNoso_level_0',
  'x8F5xyUWy9e_level_0',
  'zsNo4HB9uLZ_level_0'
 ]

In [ ]:
dataset_list

['8194nk5LbLH_level_0',
 'EU6Fwq7SyZv_level_0',
 'QUCTc6BB5sX_level_0',
 'TbHJrupSAjP_level_0',
 'X7HyMhZNoso_level_0',
 'x8F5xyUWy9e_level_0',
 'zsNo4HB9uLZ_level_0']

In [ ]:
from mmseg.apis import inference_segmentor, init_segmentor, show_result_pyplot
from mmseg.core.evaluation import get_palette
from os import listdir
import pandas as pd
from os import path
from PIL import Image
import numpy as np
from google.colab import files

In [ ]:
PATH_TO_CHECKPOINT_FILE = "/content/mmsegmentation/checkpoints/pspnet_r101-d8_512x512_160k_ade20k_20200615_100650-967c316f.pth"
PATH_TO_NET_CONFIG = "/content/mmsegmentation/configs/pspnet/pspnet_r101-d8_512x512_160k_ade20k.py"
WALL = [120,120,120]
FLOOR = [80,50,50]

model = init_segmentor(PATH_TO_NET_CONFIG, PATH_TO_CHECKPOINT_FILE , device='cuda:0')

for house in dataset_list: 
  PATH_TO_DRIVE = f"/content/drive/MyDrive/dataset/{house}.zip"
  !unzip $PATH_TO_DRIVE -d /content/
  PATH_TO_IMAGES_TO_SEGMENT = f"/content/{house}/RGB/"
  PATH_TO_IMG_OUT = f'/content/img_out_{house}/'
  PATH_TO_NUMPY_FILES = f'/content/semantic_{house}/'
  file_list = listdir(PATH_TO_IMAGES_TO_SEGMENT)
  images_list = [i for i in file_list if i.endswith('.png')]

  if not path.isdir(PATH_TO_IMG_OUT):
    for iter, image_name in enumerate(images_list): 
      print(f"Image n. {iter}")
      image = PATH_TO_IMAGES_TO_SEGMENT + image_name
      print(f"Inferecing image: {image_name}")
      result = inference_segmentor(model,image)
      out_file_name = PATH_TO_IMG_OUT + image_name
      model.show_result(image, result, out_file=out_file_name, opacity=1)

  PATH_TO_IMG_OUT_ZIP = f'/content/img_out_{house}'
  zip_name = f"{PATH_TO_IMG_OUT_ZIP}.zip"
  !zip -r $zip_name $PATH_TO_IMG_OUT_ZIP

  file_list = listdir(PATH_TO_IMG_OUT)
  images_list = [i for i in file_list if i.endswith('.png')]

  if not path.isdir(PATH_TO_NUMPY_FILES):
    !mkdir $PATH_TO_NUMPY_FILES
    for iter, image_name in enumerate(images_list): 
      print(f"Image n. {iter}")
      image_path = PATH_TO_IMG_OUT + image_name
      image = Image.open(image_path)
      data = np.asarray(image)
      #create semantic matrix, 0=wall,1=floor,2=undefined. 
      print(f"Transforming image: {image_name} in semantic")
      semantic = np.select(condlist=[np.all(data == WALL,axis = 2),np.all(data == FLOOR,axis = 2)], 
                              choicelist=[0,1],
                              default = 2)
      file_name = image_name.replace(".png", '.npy')
      np.save(PATH_TO_NUMPY_FILES + file_name, semantic)

  PATH_TO_NUMPY_FILES_ZIP = f'/content/semantic_{house}'
  semantic_zip_file_name = f"{PATH_TO_NUMPY_FILES_ZIP}.zip"
  !zip -r $semantic_zip_file_name $PATH_TO_NUMPY_FILES_ZIP

  PATH_TO_HOUSE = f"/content/{house}"
  
  !rm -rf $PATH_TO_IMG_OUT.zip
  !rm -rf $PATH_TO_IMG_OUT
  !rm -rf $PATH_TO_NUMPY_FILES.zip
  !rm -rf $PATH_TO_NUMPY_FILES
  !rm -rf $PATH_TO_HOUSE